In [27]:
import numpy as np # linear algebra
import pandas as pd # data processing, csv file I/O
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# for data preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

# NLP tools
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

# train split and fit models
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.naive_bayes import GaussianNB

# model selection
from sklearn.metrics import confusion_matrix, accuracy_score

 

[nltk_data] Downloading package stopwords to /home/arafat/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [28]:
dataset = pd.read_csv('labeled_data.csv')
pd.set_option("display.max_colwidth", None)
dataset.head()

,Unnamed: 0,count,hate_speech,offensive_language,neither,class,tweet
0,0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
1,1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!
2,2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
3,3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny
4,4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;


In [29]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24783 entries, 0 to 24782
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          24783 non-null  int64 
 1   count               24783 non-null  int64 
 2   hate_speech         24783 non-null  int64 
 3   offensive_language  24783 non-null  int64 
 4   neither             24783 non-null  int64 
 5   class               24783 non-null  int64 
 6   tweet               24783 non-null  object
dtypes: int64(6), object(1)
memory usage: 1.3+ MB


In [30]:
dataset.describe().T

,count,mean,std,min,25%,50%,75%,max
Unnamed: 0,24783.0,12681.192027,7299.553863,0.0,6372.5,12703.0,18995.5,25296.0
count,24783.0,3.243473,0.883060,3.0,3.0,3.0,3.0,9.0
hate_speech,24783.0,0.280515,0.631851,0.0,0.0,0.0,0.0,7.0
offensive_language,24783.0,2.413711,1.399459,0.0,2.0,3.0,3.0,9.0
neither,24783.0,0.549247,1.113299,0.0,0.0,0.0,0.0,9.0
class,24783.0,1.110277,0.462089,0.0,1.0,1.0,1.0,2.0


In [31]:
dt_trasformed = dataset[['class', 'tweet']]


In [33]:
dt_trasformed.head()

,class,tweet
0,2,!!! RT @mayasolovely: As a woman you shouldn't complain about cleaning up your house. &amp; as a man you should always take the trash out...
1,1,!!!!! RT @mleew17: boy dats cold...tyga dwn bad for cuffin dat hoe in the 1st place!!
2,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby4life: You ever fuck a bitch and she start to cry? You be confused as shit
3,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she look like a tranny
4,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you hear about me might be true or it might be faker than the bitch who told it to ya &#57361;


In [34]:
y = dt_trasformed.iloc[:,:-1].values

In [35]:
y

array([[2],
       [1],
       [1],
       ...,
       [1],
       [1],
       [2]])

In [6]:
type(y)

numpy.ndarray

In [25]:
y.shape

(24783, 3)

In [26]:
y

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.]])

# Encoding the Dependent Variable

In [9]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [0])], remainder='passthrough')
y = np.array(ct.fit_transform(y))

In [10]:
type(y)

numpy.ndarray

In [11]:
print(y)

[[0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


This data has been split into two variables that will be used to fit hate speech and offensive speech models.

In [12]:
y_df = pd.DataFrame(y)
y_hate = np.array(y_df[0])
y_offensive = np.array(y_df[1])

In [13]:
print(y_hate)
print(y_offensive)

[0. 0. 0. ... 0. 0. 0.]
[0. 1. 1. ... 1. 1. 0.]


# Cleaning the texts

In [14]:
corpus = []
for i in range(0, 24783):
    review = re.sub('[^a-zA-Z]', ' ', dt_trasformed['tweet'][i])
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    corpus.append(review)

In [15]:
corpus[:5]

['rt mayasolov woman complain clean hous amp man alway take trash',
 'rt mleew boy dat cold tyga dwn bad cuffin dat hoe st place',
 'rt urkindofbrand dawg rt sbabi life ever fuck bitch start cri confus shit',
 'rt c g anderson viva base look like tranni',
 'rt shenikarobert shit hear might true might faker bitch told ya']

In [16]:
cv = CountVectorizer(max_features = 2000)
X = cv.fit_transform(corpus).toarray()

#### Splitting the dataset into the Training and Test set

In [17]:
x_train, x_test, y_train, y_test = train_test_split(X, y_hate, test_size = .3, random_state = 0)

#### Finding the best model to predict hate speech
Naive Bayes

In [18]:
classifier_np = GaussianNB()
classifier_np.fit(x_train, y_train)

GaussianNB()

Logistic Regression

In [19]:
classifier_lr = LogisticRegression(random_state=0)
classifier_lr.fit(x_train, y_train)

LogisticRegression(random_state=0)

SVM Classifier

In [20]:
classifier_svm = svm.SVC()
classifier_svm.fit(x_train, y_train)

SVC()

# Making the Confusion Matrix for each model

In [21]:
# naive bayes
y_pred_np = classifier_np.predict(x_test)
cm = confusion_matrix(y_test, y_pred_np)
print(cm)

[[3289 3719]
 [ 168  259]]


In [22]:
# svm
y_pred_svm = classifier_svm.predict(x_test)
cm = confusion_matrix(y_test, y_pred_svm)
print(cm)

[[6964   44]
 [ 372   55]]


In [23]:
# logistic regression
y_pred_lr = classifier_lr.predict(x_test)
cm = confusion_matrix(y_test, y_pred_lr)
print(cm)

[[6911   97]
 [ 347   80]]


In [24]:
svm_score = accuracy_score(y_test, y_pred_svm)
lr_score = accuracy_score(y_test, y_pred_lr)
np_score = accuracy_score(y_test, y_pred_np)

print("SVM Accuracy: ", str(svm_score))
print("LR Accuracy: ", str(lr_score))
print("NB Accuracy: ", str(np_score))

SVM Accuracy:  0.9440484196368527
LR Accuracy:  0.9402824478816408
NB Accuracy:  0.4772024209818426
